In [27]:
import gensim
import pandas as pd

In [28]:
# df = pd.read_json("Cell_Phones_and_Accessories_5.json", lines=True)
# df

#the above code gave the following error

# ---------------------------------------------------------------------------
# ValueError                                Traceback (most recent call last)
# <ipython-input-4-416aa8d4b7de> in <cell line: 1>()
# ----> 1 df = pd.read_json("Cell_Phones_and_Accessories_5.json", lines=True)
#       2 df.head()

# 6 frames
# /usr/local/lib/python3.10/dist-packages/pandas/io/json/_json.py in _parse_no_numpy(self)
#    1319         if orient == "columns":
#    1320             self.obj = DataFrame(
# -> 1321                 loads(json, precise_float=self.precise_float), dtype=None
#    1322             )
#    1323         elif orient == "split":

# ValueError: No ':' found when decoding object value

#use the method used in the next cell instead

In [29]:
import json

data = []
with open("Cell_Phones_and_Accessories_5.json", "r") as file:
    for line in file:
        try:
            obj = json.loads(line)
            data.append(obj)
        except json.JSONDecodeError as e:
            print(f"Error parsing JSON: {e}")

df = pd.DataFrame(data)

In [30]:
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A30TL5EWN6DFXT,120401325X,christina,"[0, 0]",They look good and stick good! I just don't li...,4.0,Looks Good,1400630400,"05 21, 2014"
1,ASY55RVNIL0UD,120401325X,emily l.,"[0, 0]",These stickers work like the review says they ...,5.0,Really great product.,1389657600,"01 14, 2014"
2,A2TMXE2AFO7ONB,120401325X,Erica,"[0, 0]",These are awesome and make my phone look so st...,5.0,LOVE LOVE LOVE,1403740800,"06 26, 2014"
3,AWJ0WZQYMYFQ4,120401325X,JM,"[4, 4]",Item arrived in great time and was in perfect ...,4.0,Cute!,1382313600,"10 21, 2013"
4,ATX7CZYFXI1KW,120401325X,patrice m rogoza,"[2, 3]","awesome! stays on, and looks great. can be use...",5.0,leopard home button sticker for iphone 4s,1359849600,"02 3, 2013"


In [31]:
df.shape

(194439, 9)

In [32]:
df.reviewText[0]

"They look good and stick good! I just don't like the rounded shape because I was always bumping it and Siri kept popping up and it was irritating. I just won't buy a product like this again"

In [33]:
gensim.utils.simple_preprocess("They look good and stick good! I just don't like the rounded shape because I was always bumping it and Siri kept popping up and it was irritating. I just won't buy a product like this again")

['they',
 'look',
 'good',
 'and',
 'stick',
 'good',
 'just',
 'don',
 'like',
 'the',
 'rounded',
 'shape',
 'because',
 'was',
 'always',
 'bumping',
 'it',
 'and',
 'siri',
 'kept',
 'popping',
 'up',
 'and',
 'it',
 'was',
 'irritating',
 'just',
 'won',
 'buy',
 'product',
 'like',
 'this',
 'again']

In [34]:
review_text = df.reviewText.apply(gensim.utils.simple_preprocess)
review_text

0         [they, look, good, and, stick, good, just, don...
1         [these, stickers, work, like, the, review, say...
2         [these, are, awesome, and, make, my, phone, lo...
3         [item, arrived, in, great, time, and, was, in,...
4         [awesome, stays, on, and, looks, great, can, b...
                                ...                        
194434    [works, great, just, like, my, original, one, ...
194435    [great, product, great, packaging, high, quali...
194436    [this, is, great, cable, just, as, good, as, t...
194437    [really, like, it, becasue, it, works, well, w...
194438    [product, as, described, have, wasted, lot, of...
Name: reviewText, Length: 194439, dtype: object

In [37]:
model = gensim.models.Word2Vec(
    window = 10,
    min_count = 2,
    workers = 4
)

model.build_vocab(review_text, progress_per=1000)

In [38]:
model.epochs

5

In [39]:
model.train(review_text, total_examples=model.corpus_count, epochs=model.epochs)

(61508732, 83868975)

In [40]:
model.save("./word2vec.model")

In [41]:
model.wv.most_similar("bad")

[('shabby', 0.6726745963096619),
 ('terrible', 0.6714708209037781),
 ('horrible', 0.6118801236152649),
 ('awful', 0.5886332392692566),
 ('good', 0.5876023769378662),
 ('legit', 0.5819305181503296),
 ('okay', 0.5408470630645752),
 ('crappy', 0.5387846231460571),
 ('ridiculous', 0.5280092358589172),
 ('poor', 0.518295168876648)]

In [42]:
model.wv.similarity(w1="cheap", w2="inexpensive")

0.5367973

In [43]:
model.wv.similarity(w1="great", w2="good")

0.78004